# <center> <img src="../labs/img/ITESOLogo.png" alt="ITESO" width="480" height="130"> </center>
# <center> **Departamento de Electrónica, Sistemas e Informática** </center>
---
## <center> **Procesamiento de Datos Masivos** </center>
---
### <center> **Primavera 2025** </center>
---
### <center> **Lab11 Grupo Foraneos - Ejemplo de Machine Learning with Desicion trees and Support Vector Mchines** </center>

---
**Profesor**: Dr. Pablo Camarillo Ramirez

In [ ]:
import findspark
findspark.init()

#### Creacion de la conexión con el cluster de spark


In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("MLSpark-DT_SVM") \
    .master("spark://0638c7435d1d:7077") \
    .config("spark.ui.port","4040") \
    .getOrCreate()
sc = spark.sparkContext
spark.conf.set("spark.sql.shuffle.partitions", "5")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/04/29 01:51:09 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


### Creating the schema

In [3]:
from foraneos.spark_utils import SparkUtils
# Create a small dataset as a list of tuples
# Format: (label, feature_x1, feature_x2)

# Define schema for the DataFrame

schema = SparkUtils.generate_schema([("Id", "float"),("SepalLengthCm", "float"),("SepalWidthCm", "float"),("PetalLengthCm", "float"),("PetalWidthCm", "float"),\
                                    ("Species", "string")])


### Loading the Dataset

In [4]:
# Create DataFrame
df = spark \
                .read \
                .schema(schema) \
                .option("header", "true") \
                .option("mode", "dropMalformed")\
                .csv("/home/jovyan/notebooks/data/iris_dataset/iris_ds.csv")
                

### Correcting the label column to numeric values

In [5]:
from pyspark.ml.feature import StringIndexer
label_indexer = StringIndexer(inputCol="Species", outputCol="label") 
data = label_indexer.fit(df).transform(df)

In [6]:
data.show()

+----+-------------+------------+-------------+------------+-----------+-----+
|  Id|SepalLengthCm|SepalWidthCm|PetalLengthCm|PetalWidthCm|    Species|label|
+----+-------------+------------+-------------+------------+-----------+-----+
| 1.0|          5.1|         3.5|          1.4|         0.2|Iris-setosa|  0.0|
| 2.0|          4.9|         3.0|          1.4|         0.2|Iris-setosa|  0.0|
| 3.0|          4.7|         3.2|          1.3|         0.2|Iris-setosa|  0.0|
| 4.0|          4.6|         3.1|          1.5|         0.2|Iris-setosa|  0.0|
| 5.0|          5.0|         3.6|          1.4|         0.2|Iris-setosa|  0.0|
| 6.0|          5.4|         3.9|          1.7|         0.4|Iris-setosa|  0.0|
| 7.0|          4.6|         3.4|          1.4|         0.3|Iris-setosa|  0.0|
| 8.0|          5.0|         3.4|          1.5|         0.2|Iris-setosa|  0.0|
| 9.0|          4.4|         2.9|          1.4|         0.2|Iris-setosa|  0.0|
|10.0|          4.9|         3.1|          1.5|     

### Assemble the features into a single vector column

In [7]:
from pyspark.ml.feature import VectorAssembler

assembler = VectorAssembler(inputCols=["Id","SepalLengthCm","SepalWidthCm","PetalLengthCm","PetalWidthCm"], outputCol="features")
data_with_features = assembler.transform(data).select("label", "features")

### Split the data into training and test sets 80% training data and 20% testing data

In [8]:
train_df, test_df = data_with_features.randomSplit([0.8, 0.2], seed=57)

### Show the whole dataset

In [9]:
print("Original Dataset")
data_with_features.show(5)

# Print train dataset
print("train set")
train_df.show(5)

Original Dataset
+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|[1.0,5.0999999046...|
|  0.0|[2.0,4.9000000953...|
|  0.0|[3.0,4.6999998092...|
|  0.0|[4.0,4.5999999046...|
|  0.0|[5.0,5.0,3.599999...|
+-----+--------------------+
only showing top 5 rows

train set
+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|[1.0,5.0999999046...|
|  0.0|[2.0,4.9000000953...|
|  0.0|[3.0,4.6999998092...|
|  0.0|[4.0,4.5999999046...|
|  0.0|[5.0,5.0,3.599999...|
+-----+--------------------+
only showing top 5 rows



### Create a Decision Tree model

In [10]:
from pyspark.ml.classification import DecisionTreeClassifier

# Initialize and train the Decision Tree model
dt = DecisionTreeClassifier(labelCol="label", featuresCol="features")

## TRAINNING

In [11]:
dt_model = dt.fit(train_df)

# Display model summary
print("Decision Tree model summary:{0}".format(dt_model.toDebugString))

Decision Tree model summary:DecisionTreeClassificationModel: uid=DecisionTreeClassifier_59b5cdc09c72, depth=3, numNodes=7, numClasses=3, numFeatures=5
  If (feature 3 <= 2.350000023841858)
   Predict: 0.0
  Else (feature 3 > 2.350000023841858)
   If (feature 0 <= 102.5)
    If (feature 4 <= 1.8499999642372131)
     Predict: 1.0
    Else (feature 4 > 1.8499999642372131)
     Predict: 2.0
   Else (feature 0 > 102.5)
    Predict: 2.0



## PREDICTIONS

In [12]:
# Use the trained model to make predictions on the test data
predictions_DT = dt_model.transform(test_df)

# Show predictions
predictions_DT.select("features", "prediction").show()

+--------------------+----------+
|            features|prediction|
+--------------------+----------+
|[25.0,4.800000190...|       0.0|
|[26.0,5.0,3.0,1.6...|       0.0|
|[33.0,5.199999809...|       0.0|
|[38.0,4.900000095...|       0.0|
|[45.0,5.099999904...|       0.0|
|[67.0,5.599999904...|       1.0|
|[72.0,6.099999904...|       1.0|
|[73.0,6.300000190...|       1.0|
|[82.0,5.5,2.40000...|       1.0|
|[83.0,5.800000190...|       1.0|
|[86.0,6.0,3.40000...|       1.0|
|[89.0,5.599999904...|       1.0|
|[97.0,5.699999809...|       1.0|
|[101.0,6.30000019...|       2.0|
|[109.0,6.69999980...|       2.0|
|[111.0,6.5,3.2000...|       2.0|
|[113.0,6.80000019...|       2.0|
|[114.0,5.69999980...|       2.0|
|[117.0,6.5,3.0,5....|       2.0|
|[124.0,6.30000019...|       2.0|
+--------------------+----------+
only showing top 20 rows



# Support Vector machine Example

In [13]:
from pyspark.ml.classification import LinearSVC 
from pyspark.ml.classification import OneVsRest



# Initialize the LinearSVC classifier for binary 
# classification
lsvc = LinearSVC(maxIter=10, regParam=0.01)

# Set up OneVsRest classifier for multi-class 
# classification
ovr = OneVsRest(classifier=lsvc)

## Training

In [14]:
# Train the model
ovr_model = ovr.fit(train_df)

25/04/29 01:51:26 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
25/04/29 01:51:26 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.VectorBLAS


## PREDICTIONS

In [15]:
# Use the trained model to make predictions on the test data
predictions_SVM = dt_model.transform(test_df)

# Show predictions
predictions_SVM.select("features", "prediction").show()

+--------------------+----------+
|            features|prediction|
+--------------------+----------+
|[25.0,4.800000190...|       0.0|
|[26.0,5.0,3.0,1.6...|       0.0|
|[33.0,5.199999809...|       0.0|
|[38.0,4.900000095...|       0.0|
|[45.0,5.099999904...|       0.0|
|[67.0,5.599999904...|       1.0|
|[72.0,6.099999904...|       1.0|
|[73.0,6.300000190...|       1.0|
|[82.0,5.5,2.40000...|       1.0|
|[83.0,5.800000190...|       1.0|
|[86.0,6.0,3.40000...|       1.0|
|[89.0,5.599999904...|       1.0|
|[97.0,5.699999809...|       1.0|
|[101.0,6.30000019...|       2.0|
|[109.0,6.69999980...|       2.0|
|[111.0,6.5,3.2000...|       2.0|
|[113.0,6.80000019...|       2.0|
|[114.0,5.69999980...|       2.0|
|[117.0,6.5,3.0,5....|       2.0|
|[124.0,6.30000019...|       2.0|
+--------------------+----------+
only showing top 20 rows



# MODEL TESTING

In [16]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator



print("Evaluation of DT: ")
evaluator = MulticlassClassificationEvaluator(labelCol="label",
                            predictionCol="prediction")

accuracy = evaluator.evaluate(predictions_DT, 
                  {evaluator.metricName: "accuracy"})
print(f"Accuracy: {accuracy}")
precision = evaluator.evaluate(predictions_DT,
                  {evaluator.metricName: "weightedPrecision"})
print(f"Precision: {precision}")
recall = evaluator.evaluate(predictions_DT,
                  {evaluator.metricName: "weightedRecall"})
print(f"Recall: {recall}")
f1 = evaluator.evaluate(predictions_DT,
                {evaluator.metricName: "f1"})
print(f"F1 Score: {f1}") 


print("Evaluation of SVM: ")

evaluator = MulticlassClassificationEvaluator(labelCol="label",
                            predictionCol="prediction")

accuracy = evaluator.evaluate(predictions_SVM, 
                  {evaluator.metricName: "accuracy"})
print(f"Accuracy: {accuracy}")
precision = evaluator.evaluate(predictions_SVM,
                  {evaluator.metricName: "weightedPrecision"})
print(f"Precision: {precision}")
recall = evaluator.evaluate(predictions_SVM,
                  {evaluator.metricName: "weightedRecall"})
print(f"Recall: {recall}")
f1 = evaluator.evaluate(predictions_SVM,
                {evaluator.metricName: "f1"})
print(f"F1 Score: {f1}") 

Evaluation of DT: 
Accuracy: 1.0
Precision: 1.0
Recall: 1.0
F1 Score: 1.0
Evaluation of SVM: 
Accuracy: 1.0
Precision: 1.0
Recall: 1.0
F1 Score: 1.0


In [17]:
sc.stop()